In [1]:
subcases_pattern="_1"
reference_location="./"
source_name="Cyg X-1"
osa_version='OSA10.2'
nscw=3
mosaic=0
chi2_limit=1.2
systematic_fraction=0.01

#  ISGRI verification with SPI reference

simultaneous observations allow easy comparisong

one should take care to have the same effective selection

normalization need to be fitted and reported

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt


import numpy as np
import importlib
from astroquery.simbad import Simbad
from astropy import coordinates as coord
import os
import numpy as np
import glob


In [4]:
reference_location

# if reference_location  not git or http may be a problem
# dependency injection for this, if can

'./'

In [3]:

subcase=None

reference_spectrum = None

for subcase_dir in glob.glob(reference_location+"/subcases/*"+subcases_pattern+"*"):
    print("inspecting", subcase_dir)
    
    fns = [os.path.basename(fn) for fn in glob.glob(subcase_dir+"/*")]
    
    try:
        reference_spectrum = [fn for fn in fns if fn.startswith("spectrum")][0]
        scwlist = open(subcase_dir+"/ISGRI_scw_list.txt").read().split()       
        
        reference_rmf = sorted([fn for fn in fns if fn.startswith("rmf_")])[-1]
    except: continue
    
    ref_sources = np.genfromtxt(subcase_dir+"/ISGRI_sources.txt", delimiter=",", names=True)
        
    #print(fns)
    
    reference_dir=subcase_dir
    
    print("found",reference_dir, reference_spectrum, len(scwlist),len(ref_sources), ref_sources)
        
    break
        
reference_spectrum, reference_rmf

NameError: name 'glob' is not defined

In [3]:
result_table = Simbad.query_object(source_name)
source_coord = coord.SkyCoord(result_table['RA'][0], result_table['DEC'][0], unit=("hourangle", "deg"))

result_table

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object
HD 226868,19 58 21.6758,+35 12 05.782,14,14,0.023,0.029,90,A,O,2018yCat.1345....0G


In [4]:
import random

random.seed(0)
scw_pick = random.sample([
            s+"."+"001"
            for s in list(sorted(set( scwlist  ))) 
            if s.endswith("0010")
        ],nscw)


scw_list_str = ",".join(scw_pick)
scw_list_str

'105200710010.001,105200400010.001,105200650010.001'

In [5]:
import oda_api.api
import importlib
importlib.reload(oda_api)

disp = oda_api.api.DispatcherAPI(host="http://cdcihn/staging-1.2/dispatcher")

disp.get_instrument_description("isgri")




--------------
instrumet: isgri

--------------
query_name: src_query
 name: src_name,  value: test,  units: str, 
 name: RA,  value: 0.0,  units: deg, 
 name: DEC,  value: 0.0,  units: deg, 
 name: T1,  value: 2001-12-11T00:00:00.000,  units: None, 
 name: T2,  value: 2001-12-11T00:00:00.000,  units: None, 

--------------
query_name: isgri_parameters
 name: user_catalog,  value: None,  units: str, 
 name: scw_list,  value: [],  units: names_list, 
 name: radius,  value: 5.0,  units: deg, 
 name: max_pointings,  value: 50,  units: None, 
 name: osa_version,  value: None,  units: str, 
 name: E1_keV,  value: 10.0,  units: keV, 
 name: E2_keV,  value: 40.0,  units: keV, 

--------------
query_name: isgri_image_query
 product_name: isgri_image
 name: detection_threshold,  value: 0.0,  units: sigma, 
 name: image_scale_min,  value: None,  units: None, 
 name: image_scale_max,  value: None,  units: None, 

--------------
query_name: isgri_spectrum_query
 product_name: isgri_spectrum

----

In [15]:
by_lt={}

if mosaic == 1:

    for c_emin in np.linspace(20,35,2):

        image = disp.get_product(instrument="isgri", 
                         product="isgri_image", 
                         product_type="Real", 
                         osa_version=osa_version,
                         E1_keV=np.round(c_emin),
                         E2_keV=80.0,
                         scw_list=scw_list_str)

        sources=image.dispatcher_catalog_1.table[image.dispatcher_catalog_1.table['src_names']==source_name]
        source=sources[np.argmax(sources['significance'])]
        source

        d=image.mosaic_image_0.data_unit[1].data


        img=np.array(d.data)

        m_bkg=img<10
        m_bkg&=img!=0

        img_std = np.std(img[m_bkg])

        img[np.array(img)>img_std*5]=img_std*5

        plt.imshow(img)

        by_lt['%.10lg'%c_emin]=dict(
            emin=c_emin,            
            imgstd=img_std,
        )
        by_lt['%.10lg'%c_emin].update(dict([(n, source[n]) for n in source.colnames]))

        
    for lt, c in by_lt.items():
        print(lt, c['significance'],c['significance']/c['imgstd'])

In [16]:
spectrum = disp.get_product(instrument="isgri", 
                 product="isgri_spectrum", 
                 product_type="Real", 
                 osa_version=osa_version,
                 E1_keV=25.0,
                 E2_keV=80.0,
                 scw_list=scw_list_str)

105200710010.001,105200400010.001,105200650010.001
waiting for remote response, please wait run_analysis http://cdcihn/staging-1.2/dispatcher


query done succesfully!


In [17]:
specprod=[l for l in spectrum._p_list if l.meta_data['src_name'] == source_name]    

spec_fn="/tmp/isgri_spectrum_{}.fits".format(source_name)
arf_fn="/tmp/isgri_arf_{}.fits".format(source_name)
rmf_fn="/tmp/isgri_rmf_{}.fits".format(source_name)

specprod[0].write_fits_file(spec_fn)
specprod[1].write_fits_file(arf_fn)
specprod[2].write_fits_file(rmf_fn)

In [18]:

from IPython.display import Image
from IPython.display import display 
import xspec
import shutil



In [19]:
from astropy.io import fits

f=fits.open(reference_dir+"/"+reference_spectrum)
f[2].header['RESPFILE'] = 'NONE'

#f[2].header['RESPFILE']

f.writeto("reference_spec.fits", overwrite=True)

#    s_ref.response=reference_rmf
    

In [1]:
reference_spec="reference_spec.fits"
reference_rmf="reference_rmf.fits"
isgri_spec=spec_fn
isgri_arf=arf_fn
isgri_rmf=rmf_fn

NameError: name 'spec_fn' is not defined